In [1]:
!pip install --upgrade pip
!pip install pandas==1.5.3 seaborn==0.12.2 matplotlib==3.7.1

In [2]:
# Importamos las librerias necesarias.
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re

# Limpieza de datos

In [3]:
# Cargamos la data en la variable df.
df = pd.read_csv('/content/incidentes_viales.csv', delimiter=';')

/tmp/ipykernel_4306/705698297.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/crisy/universidad/fundamentosAnalitica/entregaDos/incidentes_viales.csv', delimiter=';')


In [4]:
print("La cantidad de filas que tenemos es:", df.shape[0],
      "\nY la cantidad de columnas es:", df.shape[1],
      "\n\nLas primeras columnas de nuestro data set nos arrojan la siguiente información:")

df.head()

La cantidad de filas que tenemos es: 270765 
Y la cantidad de columnas es: 18 

Las primeras columnas de nuestro data set nos arrojan la siguiente información:


,AÑO,CBML,CLASE_ACCIDENTE,DIRECCION,DIRECCION ENCASILLADA,DISEÑO,EXPEDIENTE,FECHA_ACCIDENTE,FECHA_ACCIDENTES,GRAVEDAD_ACCIDENTE,MES,NRO_RADICADO,NUMCOMUNA,BARRIO,COMUNA,LOCATION,X,Y
0,2015,1013,Caida Ocupante,CR 46 CL 43,CR 046 043 000 00000,Tramo de via,A000259731,21/10/2015 05:58:00,2015-10-21T10:58:00.000Z,Con heridos,10,1508668,10,Barrio Colón,La Candelaria,"[-75.5688011014, 6.24312304123]",834949.69,1182357.34
1,2015,1020,Choque,CR 43 A CL 29,CR 043 A 029 000 00000,Tramo de via,A000261725,05/11/2015 03:00:00,2015-11-05T08:00:00.000Z,Solo daños,11,1510621,10,San Diego,La Candelaria,"[-75.5693883283, 6.22870030622]",834880.17,1180762.02
2,2015,0205,Otro,CR 46 CL 37,CR 046 037 000 00000,Tramo de via,A000259739,21/10/2015 07:40:00,2015-10-21T12:40:00.000Z,Con heridos,10,1508691,02,La Francia,Santa Cruz,"[-75.5503911403, 6.29869502502]",837004.94,1188499.17
3,2015,0101,Atropello,CR 34 CL 112,CR 034 112 000 00000,Tramo de via,A000259752,21/10/2015 12:00:00,2015-10-21T17:00:00.000Z,Con heridos,10,1508694,01,Santo Domingo Savio No. 1,Popular,"[-75.5442195069, 6.30019383306]",837688.53,1188663.05
4,2015,1504,Choque,CR 50 CL 14,CR 050 014 000 00000,Tramo de via,A000259949,03/11/2015 06:16:00,2015-11-03T11:16:00.000Z,Solo daños,11,1510634,15,Santa Fé,Guayabal,"[-75.578180998, 6.21699243701]",833903.11,1179469.60


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270765 entries, 0 to 270764
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   AÑO                    270765 non-null  object 
 1   CBML                   252609 non-null  object 
 2   CLASE_ACCIDENTE        270759 non-null  object 
 3   DIRECCION              270765 non-null  object 
 4   DIRECCION ENCASILLADA  270374 non-null  object 
 5   DISEÑO                 269617 non-null  object 
 6   EXPEDIENTE             270655 non-null  object 
 7   FECHA_ACCIDENTE        270765 non-null  object 
 8   FECHA_ACCIDENTES       270765 non-null  object 
 9   GRAVEDAD_ACCIDENTE     270765 non-null  object 
 10  MES                    270765 non-null  int64  
 11  NRO_RADICADO           270760 non-null  object 
 12  NUMCOMUNA              270765 non-null  object 
 13  BARRIO                 251759 non-null  object 
 14  COMUNA                 257967 non-nu

Con lo anterior podemos darnos cuenta que tenemos varios datos faltantes en las siguientes columnas:

- CBML = Codigo catastral que corresponde al codigo comuna, barrio, manzana, lote catastral de un predio.
- CLASE_ACCIDENTE = Clasificación del IPAT sobre la clase de accidente de transito: choque, atropello, volcamiento, caida de ocupante, incendio, u otro (que no corresponde a las anteriores 5 clasificaciones, p. ej: sumersión)
- DIRECCION ENCASILLADA = Dirección encasillada que entrega el geocodificador
- DISEÑO = Sitio de la vía donde ocurrió el accidente: Cicloruta, Glorieta, Interseccion, Lote o Predio, Paso a Nivel, Paso Elevado, Paso Inferior, Pontón, Puente, Tramo de via, Tunel, Via peatonal
- EXPEDIENTE = Consecutivo que asigna UNE, según el orden de llegada de los expedientes para su diligenciamiento
- NRO_RADICADO = Consecutivo que asigna UNE, según el orden de llegada de los expedientes para su diligenciamiento
- BARRIO = Barrio de ocurrencia del incidente vial
- COMUNA = Denominación con la cual se identifica cada Comuna o Corregimiento. 01:Popular 02:Santa Cruz 03:Manrique 04:Aranjuez 05:Castilla 06:Doce de Octubre 07:Robledo 08:Villa Hermosa 09:Buenos Aires 10:La Candelaria 11:Laureles - Estadio 12:La América 13:San Javier 14:El Poblado 15:Guayabal 16:Belén 50:San Sebastián de Palmitas 60:San Cristobal 70:Altavista 80:San Antonio de Prado 90:Santa Elena 99:Toda la Ciudad

In [6]:
df.dtypes

AÑO                       object
CBML                      object
CLASE_ACCIDENTE           object
DIRECCION                 object
DIRECCION ENCASILLADA     object
DISEÑO                    object
EXPEDIENTE                object
FECHA_ACCIDENTE           object
FECHA_ACCIDENTES          object
GRAVEDAD_ACCIDENTE        object
MES                        int64
NRO_RADICADO              object
NUMCOMUNA                 object
BARRIO                    object
COMUNA                    object
LOCATION                  object
X                        float64
Y                        float64
dtype: object

In [7]:
# Validación de valores nulos
null = df.isnull().sum()
nullTable = pd.DataFrame({'Columna': df.columns, 'Valores nulos': null})
nullTable = nullTable.reset_index(drop=True)
nullTable

,Columna,Valores nulos
0,AÑO,0
1,CBML,18156
2,CLASE_ACCIDENTE,6
3,DIRECCION,0
4,DIRECCION ENCASILLADA,391
5,DISEÑO,1148
6,EXPEDIENTE,110
7,FECHA_ACCIDENTE,0
8,FECHA_ACCIDENTES,0
9,GRAVEDAD_ACCIDENTE,0


Ahora que no tenemos nulos, vamos a validar los valores duplicados que podemos tener.

In [8]:
# Validación de duplicados
duplicates = df.duplicated().sum()
duplicatesTable = pd.DataFrame({'Columna': df.columns, 'Duplicados': duplicates})
duplicatesTable

,Columna,Duplicados
0,AÑO,3
1,CBML,3
2,CLASE_ACCIDENTE,3
3,DIRECCION,3
4,DIRECCION ENCASILLADA,3
5,DISEÑO,3
6,EXPEDIENTE,3
7,FECHA_ACCIDENTE,3
8,FECHA_ACCIDENTES,3
9,GRAVEDAD_ACCIDENTE,3


Cómo tenemos varios valores nulos y algunos duplicados, vamos a sacarlos del data set con la siguiente celda.

In [9]:
# Eliminamos nulos
df.dropna(inplace=True)

# Eliminamos duplicados
df.drop_duplicates(inplace=True)

Ya que no tenemos duplicados, vamos a empezar a validar los valores de las columnas que tenemos.

## Año

In [10]:
df['AÑO'].unique()

array(['2015', '2014', '2016', '2017', '2018', '2019\\r', '2019', '2020',
       2020, 2015, 2014, 2016, 2017, 2018], dtype=object)

Como vemos tenemos diversos valores para el mismo año, con ello en mente vamos a convertir la columna "AÑO" a tipo de datos string y luego vamos a extraer los primeros cuatro caracteres (que representan el año).

In [11]:
df["AÑO"] = df["AÑO"].apply(lambda x : re.findall(r"[0-9]{4}",str(x))[0])
df["AÑO"].unique()

array(['2015', '2014', '2016', '2017', '2018', '2019', '2020'],
      dtype=object)

## CBML

In [12]:
df["CBML"].unique()

array(['1013', '1020', '0205', '0101', '1504', '1417', '1105', '1009',
       '1018', '0705', '0605', '1012', '0410', '0908', '0512', '0516',
       '0911', '1116', '0905', '8000', '1419', '1415', '0405', '1502',
       '0701', '1114', '0517', '1307', '0508', '0210', '0809', '1019',
       '1113', '1102', '1612', '1010', '0815', '0712', '1017', '0511',
       '0301', '1204', '0608', '0707', '1422', '0202', '1005', '1511',
       '0103', '0207', '0718', '1423', '1004', '0802', '1006', '1110',
       '1003', '0913', '0914', '0912', '1509', '0105', '1608', '0801',
       '0709', '1507', '1103', '0402', '0505', '1603', '0415', '1014',
       '1421', '1414', '1510', '0509', '0303', '0602', '0305', '1602',
       '0606', '0401', '0607', '1610', '1101', '0209', '7001', '1309',
       '1107', '1001', '1016', '1310', '0514', '0507', '1506', '1111',
       '1615', '0417', '0804', '1503', '1404', '1108', '0412', '0413',
       '0717', '0907', '0703', '1015', '6098', '0601', '1211', '1008',
      

En este caso para CBML no se denota un patron que podamos utilizar para eliminar datos que no nos aporte, así que no le realizaremos ninguna manipulación.

## CLASE_ACCIDENTE

In [13]:
df["CLASE_ACCIDENTE"].unique()

array(['Caida Ocupante', 'Choque', 'Otro', 'Atropello', 'Volcamiento',
       'Caída de Ocupante', 'Incendio'], dtype=object)

Cómo podemos ver tenemos registros con "Caída de Ocupante", vamos a reemplazar esto con "Caida de Ocupante" y vamos a pasar todas las opciones a minuscula.

In [14]:
df["CLASE_ACCIDENTE"] = df["CLASE_ACCIDENTE"].apply(lambda x : str(x).replace("Caída de Ocupante","Caida Ocupante").lower())

df["CLASE_ACCIDENTE"].unique()

array(['caida ocupante', 'choque', 'otro', 'atropello', 'volcamiento',
       'incendio'], dtype=object)

## DIRECCION

Validadremos un ejemplo de 20 registros tomados al azar para validar las direcciones.

In [15]:
df["DIRECCION"].sample(20)

251811         CR 31 CL 107
67840           CL 65 CR 78
63382           CL 14 CR 70
159720       CR 25 CL 2 Sur
21641           CL 66 CR 45
220836         CR 48 CL 104
64219           CR 64 CL 58
79959           CR 65 CL 14
117767    CL 12 Sur CR 50 G
261756          CR 65 CL 56
131354       CR 50 A CL 107
218189        CL 32 CR 65 F
194494          CR 49 CL 28
135449      CL 10 D CR 43 B
176068          CL 93 CR 23
152918          CR 46 CL 60
169085     CL 9 Sur CR 43 A
131707         CL 44 CR 9 B
166843          CR 80 CL 65
78543         CL 49 CR 81 C
Name: DIRECCION, dtype: object

Por lo que podemos ver ya se encuentra correctamente esta columna, así que no será manipulada.

## DIRECCION ENCASILLADA

Validadremos un ejemplo de 20 registros tomados al azar para validar las direcciones encasilladas.

In [16]:
df["DIRECCION ENCASILLADA"].sample(20)

236689        CR  046   051  000 00000
93665       CR  030   007 A  000 00000
188435        CL  056   027  000 00000
137774        CR  063   033  000 00000
268060        CR  052   004  000 00000
149777      CR  043 A   009  000 00000
215998        CR  052   025  000 00000
222877        CR  043   055  000 00000
70897         CR  063   059  000 00000
49852         CR  052   035  000 00000
88421       CR  045   070 A  000 00000
37747       CR  064 AA  120  000 00000
154296      DG  079 B   009  000 00000
55135         CR  062   077  000 00000
12062         CL  037   046  000 00000
53116         CR  062   050  000 00000
269972    CR  043 A   007 D  000 00000
128643       CR  049  S 007  000 00000
139955     CR  043 D  S 001  000 00000
12874        CR  052  S 014  000 00000
Name: DIRECCION ENCASILLADA, dtype: object

Por lo que podemos ver igual que en la columna anterior esta ya se encuentra correctamente, así que tampoco será manipulada.

## DISEÑO

In [17]:
df["DISEÑO"].unique()

array(['Tramo de via', 'Interseccion', 'Glorieta', 'Lote o Predio',
       'Paso a Nivel', 'Puente', 'Paso Elevado', 'Ciclo Ruta',
       'Paso Inferior', 'Via peatonal', 'Pontón', 'Tunel', 'Pont\\xF3n'],
      dtype=object)

Cómo podemos ver tenemos registros con "Pontón", vamos a reemplazar esto con "Ponton" y vamos a pasar todas las opciones a minuscula.

In [18]:
df["DISEÑO"] = df["DISEÑO"].apply(lambda x : str(x).replace("Pont\\xF3n","Pontón").lower())


df["DISEÑO"].unique()

array(['tramo de via', 'interseccion', 'glorieta', 'lote o predio',
       'paso a nivel', 'puente', 'paso elevado', 'ciclo ruta',
       'paso inferior', 'via peatonal', 'pontón', 'tunel'], dtype=object)

## EXPEDIENTE

In [19]:
df["EXPEDIENTE"].sample(20)

150306    A000254410
153215    A000373233
209599    A000841519
57262     A000092352
46603     A000188550
57279     A000092707
129768    A000595569
229482    A000186018
196364    A001008551
117379    A000997864
122127    A001064596
196611    A001009621
53142     A000783995
9422      A000382424
168919    A000596676
92310     A001150762
235622    A000094991
206087    A000853777
174719    A000114473
263636    A000377786
Name: EXPEDIENTE, dtype: object

En este caso para EXPEDIENTE no se denota un patron que podamos utilizar para eliminar datos que no nos aporte de forma similar a CBML, así que no le realizaremos ninguna manipulación.

## FECHA_ACCIDENTE y FECHA_ACCIDENTES

In [20]:
df["FECHA_ACCIDENTE"].sample(20)

95670     13/07/2020 01:30:00
122127    11/09/2019 08:30:00
21537     04/09/2018 11:20:00
15653     12/12/2017 14:30:00
263427    02/07/2015 11:30:00
114685    18/03/2019 09:38:00
61502     30/01/2017 05:00:00
152202    09/10/2015 12:00:00
230204    16/07/2014 11:30:00
223632    02/10/2014 18:15:00
110629    03/06/2018 12:30:00
187262    29/06/2018 14:44:00
89526     18/04/2017 17:30:00
157577    02/12/2015 04:30:00
170861    11/11/2017 00:10:00
63258     15/07/2014 16:20:00
203615    28/02/2018 08:10:00
31447     02/04/2019 15:40:00
6190      24/05/2017 18:00:00
183780    19/03/2020 20:30:00
Name: FECHA_ACCIDENTE, dtype: object

In [21]:
df["FECHA_ACCIDENTE"] = pd.to_datetime(df["FECHA_ACCIDENTE"], dayfirst=True)
df["FECHA_ACCIDENTE"].sample(20)

193151   2019-05-30 15:10:00
48687    2017-09-23 18:15:00
267404   2015-10-09 07:50:00
198145   2019-09-20 15:45:00
176412   2017-01-24 11:10:00
250688   2016-07-02 23:10:00
96086    2020-08-04 05:07:00
231300   2015-04-13 08:10:00
85066    2015-11-24 06:00:00
154133   2017-08-24 16:10:00
219623   2014-10-20 13:00:00
15285    2018-01-28 10:30:00
220748   2014-08-06 15:30:00
207381   2018-03-24 07:00:00
33532    2019-05-26 10:00:00
139758   2015-12-07 02:30:00
36720    2019-08-10 19:00:00
144818   2016-03-30 05:26:00
20095    2018-08-12 08:00:00
142808   2016-03-01 08:00:00
Name: FECHA_ACCIDENTE, dtype: datetime64[ns]

In [22]:
df["FECHA_ACCIDENTES"] = pd.to_datetime(df["FECHA_ACCIDENTES"], dayfirst=True)
df["FECHA_ACCIDENTES"].sample(20)

213179   2018-12-14 23:10:00+00:00
158792   2016-06-12 20:30:00+00:00
7862     2017-07-11 16:30:00+00:00
213713   2018-11-09 16:00:00+00:00
4363     2015-10-10 07:00:00+00:00
44996    2017-08-15 17:00:00+00:00
124931   2019-11-10 06:00:00+00:00
237532   2015-12-17 17:40:00+00:00
155953   2017-10-10 11:00:00+00:00
83006    2016-09-15 16:40:00+00:00
152384   2015-10-29 14:30:00+00:00
79546    2015-06-17 15:15:00+00:00
208338   2018-08-24 18:00:00+00:00
63768    2020-01-10 20:30:00+00:00
127294   2017-09-21 21:50:00+00:00
90433    2020-01-06 16:20:00+00:00
149147   2015-07-18 21:00:00+00:00
53573    2017-11-29 02:55:00+00:00
153656   2017-06-12 15:00:00+00:00
183267   2020-03-07 01:20:00+00:00
Name: FECHA_ACCIDENTES, dtype: datetime64[ns, UTC]

Dejamos ambas columnas de fecha con su tipo de datos correspondiente.

## GRAVEDAD_ACCIDENTE

In [23]:
df["GRAVEDAD_ACCIDENTE"].unique()

array(['Con heridos', 'Solo daños', 'Solo da\\xF1os', 'Con muertos'],
      dtype=object)

Cómo podemos ver tenemos registros con "Solo da\\xF1os", vamos a reemplazar esto con "Solo daños" y vamos a pasar todas las opciones a minuscula.

In [24]:
df["GRAVEDAD_ACCIDENTE"]=df["GRAVEDAD_ACCIDENTE"].apply(lambda x : str(x).replace("Solo da\\xF1os","Solo daños").lower())

df["GRAVEDAD_ACCIDENTE"].unique()

array(['con heridos', 'solo daños', 'con muertos'], dtype=object)

## MES

In [25]:
df["MES"].unique()

array([10, 11, 12,  9,  4,  3,  5,  6,  7,  8,  1,  2])

Podemos ver que los meses están correctamente, no tenemos valores extraños.

## NRO_RADICADO

In [26]:
df["NRO_RADICADO"].sample(20)

127154    1701772
126621    1700151
158727    1537385
9949      1541995
65791     1709493
117814    1673620
196284    1682229
37712     1686554
36783     1682345
10630     1599473
211795    1647292
147700    1492939
14605     1611553
191391    1667388
32484     1669102
75808     1523085
51735     1631092
160520    1564496
9915      1541442
235209    1494962
Name: NRO_RADICADO, dtype: object

En este caso para NRO_RADICADO no se denota un patron que podamos utilizar para eliminar datos que no nos aporte de forma similar a CBML y EXPEDIENTE, así que no le realizaremos ninguna manipulación.

## NUMCOMUNA

In [27]:
df["NUMCOMUNA"].unique()

array(['10', '02', '01', '15', '14', '11', '07', '06', '04', '09', '05',
       '80', '13', '08', '16', '03', '12', '70', '60', '90', 'In', 'SN',
       '0', '50', 'AU', '5', '7', '9', '4', '3', '1', '2', '8', '6',
       'Sin Inf'], dtype=object)

Como vemos tenemos datos que no corresponden a comunas realmente, así que vamos a omitir los valores como:
- "AU"
- "Sin Inf"
- "In"
- "SN"
- "0"

Adicionalmente podemos ver valores como:
- 60
- 70
- 80
- 90

Estos valores los reemplazaremos por:
- 6
- 7
- 8
- 9

Respectivamente.

In [28]:
df = df[df["NUMCOMUNA"]!="AU"]
df = df[df["NUMCOMUNA"]!="Sin Inf"]
df = df[df["NUMCOMUNA"]!="In"]
df = df[df["NUMCOMUNA"]!="SN"]
df = df[df["NUMCOMUNA"]!="0"]

In [29]:
df["NUMCOMUNA"] = df["NUMCOMUNA"].apply(lambda x : (int(x)))
df["NUMCOMUNA"] = df["NUMCOMUNA"].apply(lambda x : x/10 if x >20 else x)
df["NUMCOMUNA"] = df["NUMCOMUNA"].apply(lambda x : str(int(x)))
sorted(df["NUMCOMUNA"].unique())

['1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9']

Cómo vemos ya tenemos la información ajustada a las comunas de Medellín.

## BARRIO

In [30]:
df["BARRIO"].unique()

array(['Barrio Colón', 'San Diego', 'La Francia',
       'Santo Domingo Savio No. 1', 'Santa Fé', 'La Florida',
       'Los Conquistadores', 'La Alpujarra', 'Villa Nueva',
       'Facultad de Minas U. Nacional', 'La Esperanza',
       'Perpetuo Socorro', 'Campo Valdés No. 1', 'Miraflores',
       'Oleoducto', 'Terminal de Transporte', 'Gerona',
       'U.D. Atanasio Girardot', 'Alejandro Echavarría',
       'Cabecera San Antonio de Prado', 'Manila', 'Los Balsos No.2',
       'Moravia', 'Tenche', 'Universidad Nacional', 'Los Colores',
       'Caribe', 'Juan XXIII La Quiebra', 'Belalcázar', 'Santa Cruz',
       'Sucre', 'La Candelaria', 'El Estadio', 'Suramericana',
       'La Gloria', 'Centro Administrativo', 'Las Estancias',
       'Aures No. 2', 'Los Ángeles', 'Castilla', 'La Salle', 'La América',
       'Picacho', 'Bosques de San Pablo', 'La Aguacatala',
       'Playón de Los Comuneros', 'Estación Villa', 'La Colina',
       'Popular', 'Villa del Socorro', 'Cucaracho',
       'Santa 

Podemos ver que existen nombres de barrios donde al tildar algúna vocal generaron una secuencia de escape que representa un carácter codificado en UTF-8 en hexadecimal esto hace el nombre del barrio bastante ilegible, vamos a darle manejo a esto reemplazado el código respectivo a la vocal o letra especifica.

In [31]:
df["BARRIO"] = df["BARRIO"].apply(lambda x : str(x).lower())
df["BARRIO"] = df["BARRIO"].apply(lambda x : str(x).replace("\\xc1","a").replace("\\xe1","a").replace("á","a"))
df["BARRIO"] = df["BARRIO"].apply(lambda x : str(x).replace("\\xe9","e").replace("é","e"))
df["BARRIO"] = df["BARRIO"].apply(lambda x : str(x).replace("\\xed","i").replace("í","i"))
df["BARRIO"] = df["BARRIO"].apply(lambda x : str(x).replace("\\xf3","o").replace("ó","o"))
df["BARRIO"] = df["BARRIO"].apply(lambda x : str(x).replace("\\xfa","u").replace("ú","u"))
df["BARRIO"] = df["BARRIO"].apply(lambda x : str(x).replace("\\xf1","ñ"))

In [32]:
df["BARRIO"].unique()

array(['barrio colon', 'san diego', 'la francia',
       'santo domingo savio no. 1', 'santa fe', 'la florida',
       'los conquistadores', 'la alpujarra', 'villa nueva',
       'facultad de minas u. nacional', 'la esperanza',
       'perpetuo socorro', 'campo valdes no. 1', 'miraflores',
       'oleoducto', 'terminal de transporte', 'gerona',
       'u.d. atanasio girardot', 'alejandro echavarria',
       'cabecera san antonio de prado', 'manila', 'los balsos no.2',
       'moravia', 'tenche', 'universidad nacional', 'los colores',
       'caribe', 'juan xxiii la quiebra', 'belalcazar', 'santa cruz',
       'sucre', 'la candelaria', 'el estadio', 'suramericana',
       'la gloria', 'centro administrativo', 'las estancias',
       'aures no. 2', 'los angeles', 'castilla', 'la salle', 'la america',
       'picacho', 'bosques de san pablo', 'la aguacatala',
       'playon de los comuneros', 'estacion villa', 'la colina',
       'popular', 'villa del socorro', 'cucaracho',
       'santa 

## COMUNA

In [33]:
df["COMUNA"].unique()

array(['La Candelaria', 'Santa Cruz', 'Popular', 'Guayabal', 'El Poblado',
       'Laureles Estadio', 'Robledo', 'Doce de Octubre', 'Aranjuez',
       'Buenos Aires', 'Castilla',
       'Corregimiento de San Antonio de Prado', 'San Javier',
       'Villa Hermosa', 'Belén', 'Manrique', 'La América',
       'Corregimiento de Altavista', 'Corregimiento de San Cristóbal',
       'Corregimiento de Santa Elena', 'In',
       'Corregimiento de San Sebastián de Palmitas', 'Bel\\xE9n',
       'La Am\\xE9rica', 'Corregimiento de San Crist\\xF3bal',
       'Corregimiento de San Sebasti\\xE1n de Palmitas', 'AU'],
      dtype=object)

Para la columna "COMUNA" tenemos un caso similar al barrio, vamos a darle el mismo tratamiento.

In [34]:
df["COMUNA"] = df["COMUNA"].apply(lambda x : str(x).lower())
df["COMUNA"] = df["COMUNA"].apply(lambda x : str(x).replace("\\xc1","a").replace("\\xe1","a").replace("á","a"))

df["COMUNA"] = df["COMUNA"].apply(lambda x : str(x).replace("\\xe9","e").replace("é","e"))

df["COMUNA"] = df["COMUNA"].apply(lambda x : str(x).replace("\\xf3","o").replace("ó","o"))

df["COMUNA"].unique()

array(['la candelaria', 'santa cruz', 'popular', 'guayabal', 'el poblado',
       'laureles estadio', 'robledo', 'doce de octubre', 'aranjuez',
       'buenos aires', 'castilla',
       'corregimiento de san antonio de prado', 'san javier',
       'villa hermosa', 'belen', 'manrique', 'la america',
       'corregimiento de altavista', 'corregimiento de san cristobal',
       'corregimiento de santa elena', 'in',
       'corregimiento de san sebastian de palmitas', 'au'], dtype=object)

## LOCATION, X, Y

In [35]:
df["LOCATION"].head(10)

0    [-75.5688011014, 6.24312304123]
1    [-75.5693883283, 6.22870030622]
2    [-75.5503911403, 6.29869502502]
3    [-75.5442195069, 6.30019383306]
4     [-75.578180998, 6.21699243701]
5      [-75.561118009, 6.2047775959]
6    [-75.5810144962, 6.24576066323]
7    [-75.5752597216, 6.24685341912]
8    [-75.5624178534, 6.25341454089]
9    [-75.5931430393, 6.27330476643]
Name: LOCATION, dtype: object

In [36]:
df["X"].head(10)

0    834949.69
1    834880.17
2    837004.94
3    837688.53
4    833903.11
5    835788.30
6    833598.49
7    834235.89
8    835659.53
9    832264.64
Name: X, dtype: float64

In [37]:
df["Y"].head(10)

0    1182357.34
1    1180762.02
2    1188499.17
3    1188663.05
4    1179469.60
5    1178113.02
6    1182652.97
7    1182772.04
8    1183493.83
9    1185703.90
Name: Y, dtype: float64

Estas columnas aparentemente se encuentran bien, así que no los vamos a modificar.

## Manejo de columnas

Ahora vamos a eliminar algunas columnas que tienen datos redundantes con otras, por ejemplo:

- Direccion encasillada redunda con la columna Direccion
- Fecha accidentes redunda con la columna Fecha accidente
- Expediente redunda con la columna Nro radicado
- Comuna redunda con numcomuna

In [38]:
df.drop_duplicates(inplace = True)
df.drop(labels = ["DIRECCION ENCASILLADA", "EXPEDIENTE", "FECHA_ACCIDENTES", "NRO_RADICADO", "COMUNA"], axis=1, inplace=True)

In [39]:
df

,AÑO,CBML,CLASE_ACCIDENTE,DIRECCION,DISEÑO,FECHA_ACCIDENTE,GRAVEDAD_ACCIDENTE,MES,NUMCOMUNA,BARRIO,LOCATION,X,Y
0,2015,1013,caida ocupante,CR 46 CL 43,tramo de via,2015-10-21 05:58:00,con heridos,10,10,barrio colon,"[-75.5688011014, 6.24312304123]",834949.69,1182357.34
1,2015,1020,choque,CR 43 A CL 29,tramo de via,2015-11-05 03:00:00,solo daños,11,10,san diego,"[-75.5693883283, 6.22870030622]",834880.17,1180762.02
2,2015,0205,otro,CR 46 CL 37,tramo de via,2015-10-21 07:40:00,con heridos,10,2,la francia,"[-75.5503911403, 6.29869502502]",837004.94,1188499.17
3,2015,0101,atropello,CR 34 CL 112,tramo de via,2015-10-21 12:00:00,con heridos,10,1,santo domingo savio no. 1,"[-75.5442195069, 6.30019383306]",837688.53,1188663.05
4,2015,1504,choque,CR 50 CL 14,tramo de via,2015-11-03 06:16:00,solo daños,11,15,santa fe,"[-75.578180998, 6.21699243701]",833903.11,1179469.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
270760,2017,0913,atropello,CR 28 CL 38 F,lote o predio,2017-04-21 17:30:00,con heridos,4,9,loreto,"[-75.5559460731, 6.23260034372]",836369.48,1181189.27
270761,2017,1115,caida ocupante,CL 53 CR 69,lote o predio,2017-03-10 16:30:00,con heridos,3,11,cuarta brigada,"[-75.5851522144, 6.26044928627]",833145.10,1184279.21
270762,2017,0513,choque,CR 64 C CL 92,tramo de via,2017-03-10 17:10:00,solo daños,3,5,francisco antonio zea,"[-75.5713587099, 6.28457841509]",834679.61,1186944.12
270763,2017,0607,caida ocupante,CR 78 A CL 92,tramo de via,2017-03-10 20:00:00,con heridos,3,6,kennedy,"[-75.5831132045, 6.28957484792]",833380.07,1187500.58


## Archivo de salida

Ahora vamos generar el archivo con los datos tratados.

In [40]:
df.to_csv("dataTratada.csv", index=False)

# Construcción del modelo